In [ ]:
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [ ]:
# Load settings from config.yaml
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
# Load the ideal weights data
ideal_weights_data_path = config['ideal_weights']['data']['ideal_weights_data_path']
weights_data = pd.read_csv(ideal_weights_data_path)

# Load the indexes data
all_parameters_data_path = config['merge_and_scale_data']['data']['all_parameters_merged_data_path']
data = pd.read_csv(all_parameters_data_path, parse_dates=['Date'])

# Extract the weights with the highest total score
best_weights_row = weights_data.loc[weights_data['Total Score'].idxmax()]
weights_str = best_weights_row['Weights']

# Extract individual weights from the weights string (assuming 'w1', 'w2', 'w3', 'w4' format)
weights = {f"w{i+1}": float(val.split('=')[1]) for i, val in enumerate(weights_str.split(','))}

# Calculate the 'F' column using the extracted weights
data['F'] = round(
        (1 / (1 + (weights['w1'] * data['PR_Scaled']))) *
        (1 + (weights['w2'] * data['LAR_Scaled'])) * 
        (1 / (1 + (weights['w3'] * data['OP_Price_Scaled']))) *
        (1 + (weights['w4'] * data['AVPI_Scaled'])), 6)

# Calculate the 'Videal' column
f_max = data['F'].max() 
 
beta = config['calculate_IVS']['parametres']['beta']
alpha = config['calculate_IVS']['parametres']['alpha']

F_min = data['F'].min()
F_max = data['F'].max()

data['IVS'] = round(data['Votable Supply'] * (beta + (np.exp(alpha * ((data['F'] - F_min) / (F_max - F_min))) - 1)), 2)

data['IVS/VS'] = round(data['IVS'] / data['Votable Supply'], 2)

print('w1 -', weights['w1'])
print('w2 -', weights['w2'])
print('w3 -', weights['w3'])
print('w4 -', weights['w4'])

IVS_data_path = config['calculate_final_IVS']['data']['calculated_daily_IVS_data_path']
data.to_csv(IVS_data_path, index=False)


In [ ]:
print(data['IVS/VS'].min())
print(data['IVS/VS'].max())

In [ ]:
data = pd.read_csv(IVS_data_path)

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
f_min = data['F'].min()
f_min

In [ ]:
f_max = data['F'].max()
f_max

In [ ]:
# Resample by month and take the mean
monthly_data = data.resample('M', on='Date')[['Votable Supply', 'F', 'IVS']].mean()

monthly_data['IVS/VS'] = round(monthly_data['IVS'] / monthly_data['Votable Supply'], 2)

# Save to CSV
calculated_monthly_IVS_data_path = config['calculate_final_IVS']['data']['calculated_monthly_IVS_data_path']
monthly_data.to_csv(calculated_monthly_IVS_data_path, index=True)


In [ ]:
print(monthly_data['IVS/VS'].min())
print(monthly_data['IVS/VS'].max())

In [ ]:
monthly_data

In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 5))

# Plot the data
ax.plot(monthly_data.index, monthly_data["Votable Supply"], marker='o', color='blue', label="Votable Supply")
ax.plot(monthly_data.index, monthly_data["IVS"], marker='o', color='red', label="Ideal Votable Supply")

# Format y-axis in Millions
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.1f}M'))

# Labels and title
ax.set_xlabel("Date")
ax.set_ylabel("VS and IVS (in Millions)")
ax.set_title("VS and IVS Over Time")
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()


In [ ]:
# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 5))

# Plot the data
ax.plot(data['Date'], data["Votable Supply"], color='blue', label="Votable Supply")
ax.plot(data['Date'], data["IVS"], color='red', label="Ideal Votable Supply")

# Format y-axis in Millions
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x / 1e6:.1f}M'))

# Labels and title
ax.set_xlabel("Date")
ax.set_ylabel("VS and IVS (in Millions)")
ax.set_title("VS and IVS Over Time")
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Show the plot
plt.show()
